In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
scaler = StandardScaler()
import pandas_profiling
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score as geo
from imblearn.metrics import make_index_balanced_accuracy as iba
from sklearn.metrics import roc_curve, auc
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from pycaret.regression import *
import sqlite3
from sqlite3 import Error
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_selection import SelectKBest, mutual_info_regression, VarianceThreshold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import catboost as cb

In [2]:
df1=pd.read_csv('train.csv')

In [3]:
df2=pd.read_csv('test.csv')

In [4]:
df1.info(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          137 non-null    int64  
 1   Open Date   137 non-null    object 
 2   City        137 non-null    object 
 3   City Group  137 non-null    object 
 4   Type        137 non-null    object 
 5   P1          137 non-null    int64  
 6   P2          137 non-null    float64
 7   P3          137 non-null    float64
 8   P4          137 non-null    float64
 9   P5          137 non-null    int64  
 10  P6          137 non-null    int64  
 11  P7          137 non-null    int64  
 12  P8          137 non-null    int64  
 13  P9          137 non-null    int64  
 14  P10         137 non-null    int64  
 15  P11         137 non-null    int64  
 16  P12         137 non-null    int64  
 17  P13         137 non-null    float64
 18  P14         137 non-null    int64  
 19  P15         137 non-null    i

In [5]:
trainData=df1.drop('Id', axis=1)
testData=df2.drop('Id', axis=1)

In [6]:
trainData['Open Date'] = pd.to_datetime(trainData['Open Date'], format='%m/%d/%Y')   
testData['Open Date'] = pd.to_datetime(testData['Open Date'], format='%m/%d/%Y')

trainData['OpenDays']=""
testData['OpenDays']=""

dateLastTrain = pd.DataFrame({'Date':np.repeat(['01/01/2015'],[len(trainData)]) })
dateLastTrain['Date'] = pd.to_datetime(dateLastTrain['Date'], format='%m/%d/%Y')  
dateLastTest = pd.DataFrame({'Date':np.repeat(['01/01/2015'],[len(testData)]) })
dateLastTest['Date'] = pd.to_datetime(dateLastTest['Date'], format='%m/%d/%Y')  

trainData['OpenDays'] = dateLastTrain['Date'] - trainData['Open Date']
testData['OpenDays'] = dateLastTest['Date'] - testData['Open Date']

trainData['OpenDays'] = trainData['OpenDays'].astype('timedelta64[D]').astype(int)
testData['OpenDays'] = testData['OpenDays'].astype('timedelta64[D]').astype(int)

trainData = trainData.drop('Open Date', axis=1)
testData = testData.drop('Open Date', axis=1)

In [7]:
citygroupDummy = pd.get_dummies(trainData['City Group'])
trainData = trainData.join(citygroupDummy)

citygroupDummyTest = pd.get_dummies(testData['City Group'])
testData = testData.join(citygroupDummyTest)

trainData = trainData.drop('City Group', axis=1)
testData = testData.drop('City Group', axis=1)

In [8]:
trainData.info(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        137 non-null    object 
 1   Type        137 non-null    object 
 2   P1          137 non-null    int64  
 3   P2          137 non-null    float64
 4   P3          137 non-null    float64
 5   P4          137 non-null    float64
 6   P5          137 non-null    int64  
 7   P6          137 non-null    int64  
 8   P7          137 non-null    int64  
 9   P8          137 non-null    int64  
 10  P9          137 non-null    int64  
 11  P10         137 non-null    int64  
 12  P11         137 non-null    int64  
 13  P12         137 non-null    int64  
 14  P13         137 non-null    float64
 15  P14         137 non-null    int64  
 16  P15         137 non-null    int64  
 17  P16         137 non-null    int64  
 18  P17         137 non-null    int64  
 19  P18         137 non-null    i

In [17]:
xTrain = pd.DataFrame({'OpenDays':trainData['OpenDays'].apply(np.log),
                      'Big Cities':trainData['Big Cities'], 'Other':trainData['Other'],
                      'P2':trainData['P2'], 'P8':trainData['P8'], 'P22':trainData['P22'],
                      'P24':trainData['P24'], 'P28':trainData['P28'], 'P26':trainData['P26']})
yTrain = trainData['revenue'].apply(np.log)
xTest = pd.DataFrame({'OpenDays':testData['OpenDays'].apply(np.log),
                      'Big Cities':testData['Big Cities'], 'Other':testData['Other'],
                     'P2':testData['P2'], 'P8':testData['P8'], 'P22':testData['P22'],
                      'P24':testData['P24'], 'P28':testData['P28'], 'P26':testData['P26']})

In [25]:
cbr = cb.CatBoostRegressor(n_estimators=150)
cbr.fit(xTrain, yTrain)
pred = cbr.predict(xTest)
pred = np.exp(pred)
cbr.score(xTrain, yTrain)

Learning rate set to 0.139836
0:	learn: 0.4660570	total: 1.13ms	remaining: 168ms
1:	learn: 0.4562353	total: 2.28ms	remaining: 169ms
2:	learn: 0.4449372	total: 3.32ms	remaining: 163ms
3:	learn: 0.4353164	total: 4.4ms	remaining: 161ms
4:	learn: 0.4248500	total: 5.44ms	remaining: 158ms
5:	learn: 0.4136702	total: 13ms	remaining: 311ms
6:	learn: 0.4059482	total: 14.1ms	remaining: 289ms
7:	learn: 0.3995818	total: 15.4ms	remaining: 273ms
8:	learn: 0.3936117	total: 16.8ms	remaining: 263ms
9:	learn: 0.3866763	total: 17.9ms	remaining: 251ms
10:	learn: 0.3791909	total: 19ms	remaining: 241ms
11:	learn: 0.3728886	total: 20.3ms	remaining: 234ms
12:	learn: 0.3679323	total: 21.3ms	remaining: 225ms
13:	learn: 0.3624599	total: 22.4ms	remaining: 217ms
14:	learn: 0.3568866	total: 23.3ms	remaining: 210ms
15:	learn: 0.3499986	total: 24.2ms	remaining: 202ms
16:	learn: 0.3474656	total: 25ms	remaining: 196ms
17:	learn: 0.3415571	total: 26ms	remaining: 191ms
18:	learn: 0.3374393	total: 26.8ms	remaining: 185ms
1

0.9191057602849295

In [26]:
pred = cbr.predict(xTest)
pred = np.exp(pred)

In [27]:
testData = pd.read_csv("test.csv")
submission = pd.DataFrame({
        "Id": testData["Id"],
        "Prediction": pred
    })
submission.to_csv('submission.csv',header=True, index=False)

trainData2 = pd.DataFrame({'OpenDays':trainData['OpenDays'].apply(np.log),'revenue':trainData['revenue'].apply(np.log),
                      'Big Cities':trainData['Big Cities'], 'Other':trainData['Other'],
                      'P2':trainData['P2'], 'P8':trainData['P8'], 'P22':trainData['P22'],
                      'P24':trainData['P24'], 'P28':trainData['P28'], 'P26':trainData['P26']})

reg = setup(data=trainData2, target='revenue',imputation_type='iterative',fold_shuffle=True)

In [20]:
models = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.3435,0.2095,0.4511,-0.5252,0.0276,0.0225,0.2860
knn,K Neighbors Regressor,0.3576,0.2250,0.4660,-0.5391,0.0286,0.0235,0.0090
br,Bayesian Ridge,0.3893,0.2333,0.4707,-0.5450,0.0290,0.0256,0.0060
rf,Random Forest Regressor,0.3588,0.2140,0.4547,-0.5517,0.0279,0.0235,0.0490
ada,AdaBoost Regressor,0.3985,0.2337,0.4740,-0.5530,0.0291,0.0261,0.0150
ridge,Ridge Regression,0.4092,0.2483,0.4810,-0.5901,0.0296,0.0269,0.0060
omp,Orthogonal Matching Pursuit,0.3969,0.2406,0.4804,-0.6160,0.0296,0.0261,0.0070
en,Elastic Net,0.3987,0.2610,0.5009,-0.7033,0.0309,0.0262,0.0060
llar,Lasso Least Angle Regression,0.3987,0.2610,0.5009,-0.7033,0.0309,0.0262,0.0070
lasso,Lasso Regression,0.3987,0.2610,0.5009,-0.7033,0.0309,0.0262,0.2430
